# 1. Populate Customer Data in Azure Cosmos DB

In [1]:
# Step 1: Import dependencies, load environment variables

from azure.cosmos import CosmosClient, exceptions, PartitionKey
import os

from dotenv import load_dotenv
load_dotenv()

True

In [12]:
# Step 2: Validate that required environment variables are defined
# This requires the following properties to be defined.
#         key = to authenticate with your Azure CosmosDB account 
#    endpoint = to make requests to your Azure CosmosDB instance
#    database = the namespace for Contoso Chat app containers
#   container = the container storing your customer data

# The database and container names are hardcoded (for now)
# These names will also be used in setting up promptflow connections later
# NN-TODO:  Create environment variables for consistency and reuse 
DATABASE_NAME = 'contoso-outdoor'
CONTAINER_NAME = 'customers'

# The key and endpoint are set by the Azure provisioning step in .env
# Just read the values in and create a local CosmosClient instance.
COSMOS_ENDPOINT = os.environ["COSMOS_ENDPOINT"]
COSMOS_KEY =  os.environ["COSMOS_KEY"]
client = CosmosClient(COSMOS_ENDPOINT, credential=COSMOS_KEY)
print("✅ | Azure Cosmos DB client configured successfully")


✅ | Azure Cosmos DB client configured successfully


In [25]:
# Step 3: Check if the specified CosmosDB service contains the required database 
#         Else create it
client = CosmosClient(COSMOS_ENDPOINT, credential=COSMOS_KEY)
try:
    database = client.create_database(DATABASE_NAME)
    print("✅ | Azure Cosmos DB database CREATED: ",database.id.title())        
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(DATABASE_NAME)
    print("✅ | Azure Cosmos DB database exists: ",database.id.title())        


✅ | Azure Cosmos DB database exists:  Contoso-Outdoor


In [26]:
# Step 4: Check if the database contains the required container
#         Else create it
try:
    container = database.create_container(id=CONTAINER_NAME, partition_key=PartitionKey(path="/id"))
    print("✅ | Azure Cosmos DB container CREATED: ",container.id.title())   
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
    print("✅ | Azure Cosmos DB container exists: ",container.id.title())   
except exceptions.CosmosHttpResponseError:
    raise

✅ | Azure Cosmos DB container exists:  Customers


In [31]:
# Step 5: Read customer data records from the specified folder (should be JSON files)
#         Iterate through JSON files in the folder (each representing a customer record)
#         Upsert each record into the CosmosDB container
import os
import json
import glob
path = '../data/customer_info'
 
for filename in glob.glob(os.path.join(path, '*.json')):
    with open(filename) as file:
        customer = json.load(file)
        container.upsert_item(customer)
        print('Upserted item with id: {0}'.format(customer['id']))

print("✅ | Updated Azure Cosmos DB container from data in: ",path)  


Upserted item with id 3
Upserted item with id 6
Upserted item with id 10
Upserted item with id 11
Upserted item with id 12
Upserted item with id 7
Upserted item with id 8
Upserted item with id 4
Upserted item with id 2
Upserted item with id 9
Upserted item with id 1
Upserted item with id 5
✅ | Updated Azure Cosmos DB container from data in:  ../data/customer_info


In [38]:
# Step 6: Retrieve and list container items to validate data was inserted correctly
items = list(container.read_all_items(max_item_count=10))
print("✅ | Printing the ",len(items),"items from container for validation")  
for item in items:
    print(item)


✅ | Printing the  12 items from container for validation
{'id': '3', 'firstName': 'Michael', 'lastName': 'Johnson', 'age': 45, 'email': 'michaelj@example.com', 'phone': '555-555-1212', 'address': '789 Elm St,  Smallville USA, 34567', 'membership': 'Base', 'orders': [{'id': 20, 'productId': 5, 'quantity': 2, 'total': 120.0, 'date': '2/28/2023', 'name': 'BaseCamp Folding Table', 'unitprice': 60.0, 'category': 'Camping Tables', 'brand': 'CampBuddy', 'description': "CampBuddy's BaseCamp Folding Table is an adventurer's best friend. Lightweight yet powerful, the table is a testament to fun-meets-function and will elevate any outing to new heights. Crafted from resilient, rust-resistant aluminum, the table boasts a generously sized 48 x 24 inches tabletop, perfect for meal times, games and more. The foldable design is a godsend for on-the-go explorers. Adjustable legs rise to the occasion to conquer uneven terrains and offer height versatility, while the built-in handle simplifies transporta